In [1]:
import sys
from pathlib import Path

# Get project root (2 levels up from current notebook)
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

In [2]:
from src.data_ingestion.auth import Auth
from src.data_ingestion.spotify_fetch_artist import fetch_artist  
from src.data_ingestion.spotify_fetch_playlist import fetch_top_artist_playlists 

In [3]:
artist_id = "1ukmGETCwXTbgrTrkRDnmn"  
artist_data = fetch_artist(artist_id)

print(artist_data["name"])
print(artist_data["followers"]["total"])
print(artist_data["popularity"])

2025-12-26 23:30:35 - INFO - Attempting to get Spotify access token
2025-12-26 23:30:36 - INFO - Successfully obtained access token
2025-12-26 23:30:36 - INFO - Fetching artist data for artist_id: 1ukmGETCwXTbgrTrkRDnmn
2025-12-26 23:30:36 - INFO - Successfully fetched artist data for 1ukmGETCwXTbgrTrkRDnmn


Nathaniel Bassey
3683426
63


In [4]:
artist_data

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1ukmGETCwXTbgrTrkRDnmn'},
 'followers': {'href': None, 'total': 3683426},
 'genres': ['gospel', 'worship', 'afrogospel'],
 'href': 'https://api.spotify.com/v1/artists/1ukmGETCwXTbgrTrkRDnmn',
 'id': '1ukmGETCwXTbgrTrkRDnmn',
 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb11de9e26d17433c5d960c15f',
   'height': 640,
   'width': 640},
  {'url': 'https://i.scdn.co/image/ab6761610000517411de9e26d17433c5d960c15f',
   'height': 320,
   'width': 320},
  {'url': 'https://i.scdn.co/image/ab6761610000f17811de9e26d17433c5d960c15f',
   'height': 160,
   'width': 160}],
 'name': 'Nathaniel Bassey',
 'popularity': 63,
 'type': 'artist',
 'uri': 'spotify:artist:1ukmGETCwXTbgrTrkRDnmn'}

In [6]:
artist_top_tracks = fetch_top_artist_playlists(artist_id, "NG")

2025-12-26 23:42:05 - INFO - Attempting to get Spotify access token
2025-12-26 23:42:05 - INFO - Successfully obtained access token
2025-12-26 23:42:05 - INFO - Fetching artist data for artist_id: 1ukmGETCwXTbgrTrkRDnmn
2025-12-26 23:42:06 - INFO - Successfully fetched top artist data for 1ukmGETCwXTbgrTrkRDnmn
